In [ ]:
# كود التجهيز الأولي لربط جوجل درايف ورفع ملفات API
# -*- coding: utf-8 -*-
# استيراد مكتبة جوجل درايف
from google.colab import drive
# استيراد مكتبة التعامل مع الملفات في جوجل كولاب
from google.colab import files

# تثبيت أي مكتبات ضرورية
!pip install kaggle

# طلب ربط الحساب وتحديد نقطة الوصول
drive.mount('/content/drive')


# --- رفع ملفات API ---

# 1. رفع ملف Kaggle API
print('Please upload the "kaggle.json" file:')
files.upload()

# 2. رفع ملف Unsplash API (تمت إضافة هذا الجزء)
print('\nPlease upload the "Unsplash.json" file:')
files.upload()


# --- إعداد Kaggle API ---

# إنشاء مجلد مخفي باسم .kaggle في الدليل الرئيسي
!mkdir -p ~/.kaggle

# نسخ ملف كاجل الذي تم رفعه إلى المجلد الجديد
!cp kaggle.json ~/.kaggle/

# تعيين أذونات القراءة والكتابة للمستخدم فقط
!chmod 600 ~/.kaggle/kaggle.json

print("\n--- Setup Complete. Both Kaggle and Unsplash API files are ready. ---")

In [ ]:
# نموذجي التدريبي الاساسي فيه خاصية البدء من جديد او الاستكمال
# لد تم استكمال النموذج والاوزان بدقة 87% حيث تم تخزينها في ملف افضل الاوزان

# -*- coding: utf-8 -*-
# --- الكود المتكامل والذكي: للبدء لأول مرة أو لاستئناف التدريب ---

# استيراد المكتبات الضرورية
import tensorflow as tf
import kagglehub
import os
import shutil
import time
import keras

# طباعة إصدارات المكتبات للتأكد من سلامة البيئة
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")

# =================================================================
# الخطوة 1: تحميل وتنظيف البيانات
# هذه الخطوة ضرورية في كل جلسة جديدة لأن مجلد /content/ مؤقت
# =================================================================
print("\nGetting dataset path from Kaggle Hub...")
try:
    source_dataset_path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")
except Exception as e:
    print(f"\n!! ERROR !!: Could not access Kaggle. Details: {e}")
    print("Please ensure your kaggle.json is uploaded and you've run the initial setup cell.")
    raise e

original_data_dir = os.path.join(source_dataset_path, 'PetImages')
cleaned_data_dir = '/content/PetImages_Cleaned'

if os.path.exists(cleaned_data_dir):
    shutil.rmtree(cleaned_data_dir)
os.makedirs(cleaned_data_dir, exist_ok=True)

print("\nCopying and cleaning data...")
num_skipped = 0
num_copied = 0
for class_name in ['Cat', 'Dog']:
    os.makedirs(os.path.join(cleaned_data_dir, class_name), exist_ok=True)
    source_class_dir = os.path.join(original_data_dir, class_name)
    if not os.path.isdir(source_class_dir): continue
    for fname in os.listdir(source_class_dir):
        source_fpath = os.path.join(source_class_dir, fname)
        try:
            img_bytes = tf.io.read_file(source_fpath)
            decoded_img = tf.io.decode_image(img_bytes, channels=3)
            if len(decoded_img.shape) != 3: raise ValueError("Image not 3D")
            dest_fpath = os.path.join(cleaned_data_dir, class_name, fname)
            shutil.copyfile(source_fpath, dest_fpath)
            num_copied += 1
        except Exception:
            num_skipped += 1
print(f"Finished. Copied {num_copied} valid files. Skipped {num_skipped} corrupt files.")


# =================================================================
# الخطوة 2: إنشاء مجموعات البيانات
# =================================================================
BATCH_SIZE = 32
IMG_SIZE = (180, 180)
print("\nCreating datasets from the CLEANED directory...")
train_dataset = tf.keras.utils.image_dataset_from_directory(
    cleaned_data_dir, validation_split=0.2, subset="training", seed=123, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    cleaned_data_dir, validation_split=0.2, subset="validation", seed=123, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)

# =================================================================
# الخطوة 3: بناء النموذج أو تحميله (المنطق الذكي)
# =================================================================
checkpoint_dir = "/content/drive/MyDrive/ML_Cat_Dog_Project/checkpoints"
last_model_path = os.path.join(checkpoint_dir, "last_model.keras")

# التحقق من وجود نموذج محفوظ
if os.path.exists(last_model_path):
    # --- مسار الاستئناف ---
    print("\n--- Found a saved model. Loading it to resume training... ---")
    model = tf.keras.models.load_model(last_model_path)
    print("Model loaded successfully.")
else:
    # --- مسار البدء من الصفر ---
    print("\n--- No saved model found. Building a new model from scratch... ---")
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=IMG_SIZE + (3,)),
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.Conv2D(32, 3, activation="relu"),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation="relu"),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(128, 3, activation="relu"),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    print("New model built and compiled.")

model.summary()

# =================================================================
# الخطوة 4: إعداد الحفظ والتدريب
# =================================================================
os.makedirs(checkpoint_dir, exist_ok=True)
best_model_path = os.path.join(checkpoint_dir, "best_model.keras")

best_model_callback = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_path, save_best_only=True, monitor="val_accuracy", verbose=1)
last_model_callback = tf.keras.callbacks.ModelCheckpoint(filepath=last_model_path, save_best_only=False)

# حدد العدد الإجمالي للدورات التي تريد الوصول إليها
# إذا كنت قد أكملت 4 وتريد الوصول إلى 15، يمكنك تدريب 11 دورة إضافية
epochs = 15
print(f"\n--- Starting/Resuming training for {epochs} epochs ---")

start_time = time.time()
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset,
    callbacks=[best_model_callback, last_model_callback]
)
end_time = time.time()
training_time = end_time - start_time

print(f"\n--- Training Finished ---")
print(f"Total time for this session: {training_time:.2f} seconds")

In [ ]:
# اختبار من ملفات محلية من ملفاتي على قوقل درايف
# -*- coding: utf-8 -*-
# --- كود التقييم الشامل (صور فردية + تقرير نهائي) ---

# استيراد المكتبات اللازمة
import tensorflow as tf
import numpy as np
import os
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

# =================================================================
# الخطوة 1: تحميل أفضل نموذج وتحديد المسارات
# =================================================================
print("--- Loading the best performing model... ---")
best_model_path = "/content/drive/MyDrive/ML_Cat_Dog_Project/checkpoints/best_model.keras"
model = tf.keras.models.load_model(best_model_path)

test_dir = "/content/drive/MyDrive/ML_Cat_Dog_Project/test_images"
cat_test_dir = os.path.join(test_dir, "cats")
dog_test_dir = os.path.join(test_dir, "dogs")

# تحديد أسماء الفئات بالترتيب الصحيح (أبجديًا كما يفعل كيرس)
# نفترض أن المجلدات هي 'cats' و 'dogs' ليكون الترتيب ['cats', 'dogs']
class_names = sorted(os.listdir(test_dir))
print(f"Classes found and ordered as: {class_names}")

# =================================================================
# الخطوة 2: عرض التنبؤات الفردية وتجميع النتائج
# =================================================================
y_true = []
y_pred = []

# --- تم تصحيح تعريف الدالة هنا ---
def predict_local_image(image_path, true_label_index):
    """
    تعالج صورة واحدة، تعرضها مع تنبؤها، وتُرجع النتائج
    """
    try:
        img = tf.keras.utils.load_img(image_path, target_size=IMG_SIZE)
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        prediction = model.predict(img_array)
        score = prediction[0][0]
        predicted_class_index = 1 if score > 0.5 else 0

        # عرض الصورة والتنبؤ
        plt.figure(figsize=(4, 4))
        plt.imshow(img)
        plt.axis('off')

        confidence = 100 * score if predicted_class_index == 1 else 100 * (1 - score)
        # استخدام true_label_index لتحديد التسمية الحقيقية
        title = f"Prediction: {class_names[predicted_class_index]} ({confidence:.2f}%)\n"
        title += f"Actual: {class_names[true_label_index]}"
        plt.title(title)
        plt.show()

        y_true.append(true_label_index)
        y_pred.append(predicted_class_index)

    except Exception as e:
        print(f"Could not process image: {image_path}\nError: {e}")

# --- اختبار صور القطط ---
print("\n\n--- Testing Cat Images ---")
if os.path.exists(cat_test_dir):
    for image_file in os.listdir(cat_test_dir):
        # الآن الاستدعاء متوافق مع تعريف الدالة
        predict_local_image(os.path.join(cat_test_dir, image_file), true_label_index=0) # 0 for Cat
else:
    print(f"Directory not found: {cat_test_dir}")

# --- اختبار صور الكلاب ---
print("\n\n--- Testing Dog Images ---")
if os.path.exists(dog_test_dir):
    for image_file in os.listdir(dog_test_dir):
        # الآن الاستدعاء متوافق مع تعريف الدالة
        predict_local_image(os.path.join(dog_test_dir, image_file), true_label_index=1) # 1 for Dog
else:
    print(f"Directory not found: {dog_test_dir}")

# =================================================================
# الخطوة 3: بناء وحفظ التقرير النهائي (يظهر في نهاية المخرجات)
# =================================================================
print("\n\n\n" + "="*50)
print("             FINAL PERFORMANCE REPORT")
print("="*50)

accuracy = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred, target_names=class_names, zero_division=0)

timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
report_string = f"""
Test Time: {timestamp}
Test Data Source: {test_dir}
Number of Test Images: {len(y_true)}
-------------------------------------------------

### OVERALL ACCURACY ###
Accuracy: {accuracy:.2%}

-------------------------------------------------

### CLASSIFICATION REPORT ###
{class_report}

-------------------------------------------------

### TEXT-BASED CONFUSION MATRIX ###

                 Predicted Cat | Predicted Dog
               ----------------|----------------
    Actual Cat |      {cm[0, 0]:<7} |      {cm[0, 1]:<7}
    Actual Dog |      {cm[1, 0]:<7} |      {cm[1, 1]:<7}

-------------------------------------------------
Matrix Details:
- True Cats (TN): {cm[0, 0]}
- False Dogs (FP): {cm[0, 1]}  (Cat predicted as Dog)
- False Cats (FN): {cm[1, 0]}  (Dog predicted as Cat)
- True Dogs (TP): {cm[1, 1]}
=================================================
"""

report_dir = "/content/drive/MyDrive/ML_Cat_Dog_Project/test_reports"
os.makedirs(report_dir, exist_ok=True)
report_filename = f"test_report_{time.strftime('%Y%m%d-%H%M%S')}.txt"
report_path = os.path.join(report_dir, report_filename)
with open(report_path, 'w') as f:
    f.write(report_string)

print(f"\n---> Performance report saved to: {report_path} <---")
print(report_string)